In [3]:
# !pip install pandas
# !pip install openpyxl
# !pip install matplotlib
# !pip install scipy
# !pip install requests
# !pip install dask

In [1]:
import pandas as pd
import glob
from datetime import datetime
import numpy as np
#import matplotlib.pyplot as plt
#from scipy import stats
import ast
import os
import requests
import re

In [2]:
#обработка разовая
def integration_date(file):
    path = f'{file}'
    filenames = glob.glob(path + "/*.csv")
    
    # Создаем два списка для каждой категории
    in_app_events_dfs = []
    fraud_post_inapps_dfs = []
    
    # Используем эти переменные в начальных значениях
    in_app_events_result = pd.DataFrame()
    fraud_post_inapps_result = pd.DataFrame()

    # Проверка существования папки, если нет, то создаем
    subfolder = "result_data"
    output_path = os.path.join(file, subfolder)  # Обновленный путь для сохранения файлов
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    # Проходим по всем файлам и добавляем их данные в соответствующий список
    for filename in filenames:
        if 'in-app-events' in filename:
            df = pd.read_csv(filename)
            in_app_events_dfs.append(df)
        elif 'fraud-post-inapps' in filename:
            df = pd.read_csv(filename)
            fraud_post_inapps_dfs.append(df)


    
    try:
        # Используем pd.concat для каждой категории
        if in_app_events_dfs:
            in_app_events_result = pd.concat(in_app_events_dfs, axis=0, ignore_index=True)
        else:
            raise pd.errors.EmptyDataError("No 'in-app-events' data to concatenate")
    except pd.errors.EmptyDataError as e:
        print(e)
    
    try:
        if fraud_post_inapps_dfs:
            fraud_post_inapps_result = pd.concat(fraud_post_inapps_dfs, axis=0, ignore_index=True)
        else:
            raise pd.errors.EmptyDataError("No 'fraud-post-inapps' data to concatenate")
    except pd.errors.EmptyDataError as e:
        print(e)
    
    print(len(in_app_events_result))
    print(len(fraud_post_inapps_result))
    
    # Проверяем наличие столбца 'AppsFlyer ID' в DataFrame in_app_events_result
    if 'AppsFlyer ID' in in_app_events_result.columns:
           
        in_app_events_result = in_app_events_result.sort_values(by='Event Time', ascending=True)


        drop_duplicates_ = in_app_events_result.drop_duplicates(subset=['AppsFlyer ID'])

        # Группируем данные
        group_unique_ = drop_duplicates_.groupby(['Platform','Media Source', 'Country Code'])['Attributed Touch Type'].count().reset_index()
        group_ = in_app_events_result.groupby(['Platform','Media Source', 'Country Code'])['Attributed Touch Type'].count().reset_index()
        
        # Сохраняем результаты в новые xlsx-файлы
        with pd.ExcelWriter(f"{output_path}/in_app_events_result.xlsx") as writer:
            # Сохраняем данные без мошеннических идентификаторов на первом листе
            in_app_events_result.to_excel(writer, sheet_name='in_app_events_result', index=False)

            
            # Сохраняем результат группировки на втором листе
            group_unique_.to_excel(writer, sheet_name='Grouped Data (unique)', index=False)
            group_.to_excel(writer, sheet_name='Grouped Data', index=False)    
        fraud_post_inapps_result.to_excel(f"{output_path}/fraud_post_inapps_result.xlsx", index=False)

    else:
        print("No 'Platform...' column in 'in_app_events_result' DataFrame")



    
    # Определяем значения AppsFlyer ID из fraud_post_inapps_result
    fraud_apps_ids = fraud_post_inapps_result['AppsFlyer ID'].unique() if not fraud_post_inapps_result.empty else []

    # Фильтруем строки в in_app_events_result передавая значения переменной sorted_date        
    in_app_events_result = in_app_events_result.sort_values(by='Event Time', ascending=True)
    
    # Фильтруем строки в in_app_events_result, исключая те, где AppsFlyer ID совпадает с fraud_apps_ids
    in_app_events_result_no_fraud = in_app_events_result[~in_app_events_result['AppsFlyer ID'].isin(fraud_apps_ids)]

    # Создаем переменную где удаляем дубли для отедльного листа в будущем файле in_app_events_result_no_fraud
    drop_duplicates = in_app_events_result_no_fraud.drop_duplicates(subset=['AppsFlyer ID'])
    
    # Группируем данные
    group_unique = drop_duplicates.groupby(['Platform','Media Source', 'Country Code'])['Attributed Touch Type'].count().reset_index()
    group = in_app_events_result_no_fraud.groupby(['Platform','Media Source', 'Country Code'])['Attributed Touch Type'].count().reset_index()
    
    # Создаем объект ExcelWriter для записи в один файл Excel
    with pd.ExcelWriter(f"{output_path}/in_app_events_result_no_fraud.xlsx") as writer:
        # Сохраняем данные без мошеннических идентификаторов на первом листе
        in_app_events_result_no_fraud.to_excel(writer, sheet_name='No Fraud Data', index=False)

        # Сохраняем результат группировки на втором листе
        group_unique.to_excel(writer, sheet_name='Grouped Data (unique)', index=False)
        
        group.to_excel(writer, sheet_name='Grouped Data', index=False)


    

    # Создаем переменную in_app_events_result_highlight_fraud и копируем в нее данные in_app_events_result
    in_app_events_result_highlight_fraud = in_app_events_result.copy()

    # Фильтруем строки в in_app_events_result_highlight_fraud передавая значения переменной sorted_date        
    in_app_events_result_highlight_fraud = in_app_events_result_highlight_fraud.sort_values(by='Event Time', ascending=True)

    # Отмечаем а не удаляем фроды в отдельном поле
    in_app_events_result_highlight_fraud["Fraud"] = in_app_events_result_highlight_fraud['AppsFlyer ID'].isin(fraud_apps_ids)

    # Создаем переменную где удаляем дубли для отедльного листа в будущем файле drop_duplicates_highlight_fraud
    drop_duplicates_highlight_fraud = in_app_events_result_highlight_fraud.drop_duplicates(subset=['AppsFlyer ID'])
    
    # Группируем данные в лист где уникальные
    group_unique_highlight_fraud = drop_duplicates_highlight_fraud.groupby(['Platform','Media Source', 'Country Code', 'Fraud'])['Attributed Touch Type'].count().reset_index()
    # Группируем данные в лист где не уникальные
    group_highlight_fraud = in_app_events_result_highlight_fraud.groupby(['Platform','Media Source', 'Country Code', 'Fraud'])['Attributed Touch Type'].count().reset_index()
    
    # Создаем объект ExcelWriter для записи в один файл Excel
    with pd.ExcelWriter(f"{output_path}/in_app_events_result_highlight_fraud.xlsx") as writer:
        # Сохраняем данные без мошеннических идентификаторов на первом листе
        in_app_events_result_highlight_fraud.to_excel(writer, sheet_name='highlight_fraud', index=False)

        # Сохраняем результат группировки на втором листе
        group_unique_highlight_fraud.to_excel(writer, sheet_name='Grouped Data (unique)', index=False)
        
        group_highlight_fraud.to_excel(writer, sheet_name='Grouped Data', index=False)

name_file = "G:/.shortcut-targets-by-id/1-0y0YvUdxNOM5ZnhM4sE0iJVter1OKIh/тест/AF/01.2025/Marathonbet_2025-02-05"
#name_file = "G:/.shortcut-targets-by-id/1-0y0YvUdxNOM5ZnhM4sE0iJVter1OKIh/тест/AF/12.2024/Magnit__2025-01-20"
integration_date(name_file)



C:\Users\serge\AppData\Local\Temp\ipykernel_21188\3146588695.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  in_app_events_result = pd.concat(in_app_events_dfs, axis=0, ignore_index=True)


1
0


In [13]:
a = 'asdds'
print(len(a)%2)
a = a + '_'
print(a)
print(a[:2],a[2:])


1
asdds_
as dds_


In [1]:
#Обработка всех выгрузок в папке
def integration_date(file):
    path = f'{file}'
    filenames = glob.glob(path + "/*.csv")
    
    # Создаем два списка для каждой категории
    in_app_events_dfs = []
    fraud_post_inapps_dfs = []
    
    # Используем эти переменные в начальных значениях
    in_app_events_result = pd.DataFrame()
    fraud_post_inapps_result = pd.DataFrame()

    # Проверка существования папки, если нет, то создаем
    subfolder = "result_data"
    output_path = os.path.join(file, subfolder)  # Обновленный путь для сохранения файлов
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    # Проходим по всем файлам и добавляем их данные в соответствующий список
    for filename in filenames:
        if 'in-app-events' in filename:
            df = pd.read_csv(filename)
            in_app_events_dfs.append(df)
        elif 'fraud-post-inapps' in filename:
            df = pd.read_csv(filename)
            fraud_post_inapps_dfs.append(df)


    
    try:
        # Используем pd.concat для каждой категории
        if in_app_events_dfs:
            in_app_events_result = pd.concat(in_app_events_dfs, axis=0, ignore_index=True)
        else:
            raise pd.errors.EmptyDataError("No 'in-app-events' data to concatenate")
    except pd.errors.EmptyDataError as e:
        print(e)
    
    try:
        if fraud_post_inapps_dfs:
            fraud_post_inapps_result = pd.concat(fraud_post_inapps_dfs, axis=0, ignore_index=True)
        else:
            raise pd.errors.EmptyDataError("No 'fraud-post-inapps' data to concatenate")
    except pd.errors.EmptyDataError as e:
        print(e)
    
    print(len(in_app_events_result))
    print(len(fraud_post_inapps_result))
    
    # Проверяем наличие столбца 'AppsFlyer ID' в DataFrame in_app_events_result
    if 'AppsFlyer ID' in in_app_events_result.columns:
           
        in_app_events_result = in_app_events_result.sort_values(by='Event Time', ascending=True)


        drop_duplicates_ = in_app_events_result.drop_duplicates(subset=['AppsFlyer ID'])

        # Группируем данные
        group_unique_ = drop_duplicates_.groupby(['Platform','Media Source', 'Country Code'])['Attributed Touch Type'].count().reset_index()
        group_ = in_app_events_result.groupby(['Platform','Media Source', 'Country Code'])['Attributed Touch Type'].count().reset_index()
        
        # Сохраняем результаты в новые xlsx-файлы
        with pd.ExcelWriter(f"{output_path}/in_app_events_result.xlsx") as writer:
            # Сохраняем данные без мошеннических идентификаторов на первом листе
            in_app_events_result.to_excel(writer, sheet_name='in_app_events_result', index=False)

            
            # Сохраняем результат группировки на втором листе
            group_unique_.to_excel(writer, sheet_name='Grouped Data (unique)', index=False)
            group_.to_excel(writer, sheet_name='Grouped Data', index=False)    
        fraud_post_inapps_result.to_excel(f"{output_path}/fraud_post_inapps_result.xlsx", index=False)

    else:
        print("No 'Platform...' column in 'in_app_events_result' DataFrame")


    # Определяем значения AppsFlyer ID из fraud_post_inapps_result
    fraud_apps_ids = fraud_post_inapps_result['AppsFlyer ID'].unique() if not fraud_post_inapps_result.empty else []

    # Фильтруем строки в in_app_events_result передавая значения переменной sorted_date        
    in_app_events_result = in_app_events_result.sort_values(by='Event Time', ascending=True)
    
    # Фильтруем строки в in_app_events_result, исключая те, где AppsFlyer ID совпадает с fraud_apps_ids
    in_app_events_result_no_fraud = in_app_events_result[~in_app_events_result['AppsFlyer ID'].isin(fraud_apps_ids)]

    # Создаем переменную где удаляем дубли для отедльного листа в будущем файле in_app_events_result_no_fraud
    drop_duplicates = in_app_events_result_no_fraud.drop_duplicates(subset=['AppsFlyer ID'])
    
    # Группируем данные
    group_unique = drop_duplicates.groupby(['Platform','Media Source', 'Country Code'])['Attributed Touch Type'].count().reset_index()
    group = in_app_events_result_no_fraud.groupby(['Platform','Media Source', 'Country Code'])['Attributed Touch Type'].count().reset_index()
    
    # Создаем объект ExcelWriter для записи в один файл Excel
    with pd.ExcelWriter(f"{output_path}/in_app_events_result_no_fraud.xlsx") as writer:
        # Сохраняем данные без мошеннических идентификаторов на первом листе
        in_app_events_result_no_fraud.to_excel(writer, sheet_name='No Fraud Data', index=False)

        # Сохраняем результат группировки на втором листе
        group_unique.to_excel(writer, sheet_name='Grouped Data (unique)', index=False)
        
        group.to_excel(writer, sheet_name='Grouped Data', index=False)


    

    # Создаем переменную in_app_events_result_highlight_fraud и копируем в нее данные in_app_events_result
    in_app_events_result_highlight_fraud = in_app_events_result.copy()

    # Фильтруем строки в in_app_events_result_highlight_fraud передавая значения переменной sorted_date        
    in_app_events_result_highlight_fraud = in_app_events_result_highlight_fraud.sort_values(by='Event Time', ascending=True)

    # Отмечаем а не удаляем фроды в отдельном поле
    in_app_events_result_highlight_fraud["Fraud"] = in_app_events_result_highlight_fraud['AppsFlyer ID'].isin(fraud_apps_ids)

    # Создаем переменную где удаляем дубли для отедльного листа в будущем файле drop_duplicates_highlight_fraud
    drop_duplicates_highlight_fraud = in_app_events_result_highlight_fraud.drop_duplicates(subset=['AppsFlyer ID'])
    
    # Группируем данные в лист где уникальные
    group_unique_highlight_fraud = drop_duplicates_highlight_fraud.groupby(['Platform','Media Source', 'Country Code', 'Fraud'])['Attributed Touch Type'].count().reset_index()
    # Группируем данные в лист где не уникальные
    group_highlight_fraud = in_app_events_result_highlight_fraud.groupby(['Platform','Media Source', 'Country Code', 'Fraud'])['Attributed Touch Type'].count().reset_index()
    
    # Создаем объект ExcelWriter для записи в один файл Excel
    with pd.ExcelWriter(f"{output_path}/in_app_events_result_highlight_fraud.xlsx") as writer:
        # Сохраняем данные без мошеннических идентификаторов на первом листе
        in_app_events_result_highlight_fraud.to_excel(writer, sheet_name='highlight_fraud', index=False)

        # Сохраняем результат группировки на втором листе
        group_unique_highlight_fraud.to_excel(writer, sheet_name='Grouped Data (unique)', index=False)
        
        group_highlight_fraud.to_excel(writer, sheet_name='Grouped Data', index=False)


def process_folders(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for directory in dirs:
            folder_path = os.path.join(root, directory)
            integration_date(folder_path)

# Вызов функции
root_folder = "G:/.shortcut-targets-by-id/1-0y0YvUdxNOM5ZnhM4sE0iJVter1OKIh/тест/AF/09.2024/"
process_folders(root_folder)


NameError: name 'os' is not defined

In [5]:
#окно атрибцуии
def process_csv(input_path, output_path, filter_value_days, file):
    # Чтение xlsx-файла и создание DataFrame
    df = pd.read_excel(input_path, parse_dates=['Install Time', 'Event Time'])
    
    # Проверка существования папки, если нет, то создаем
    subfolder = "WA"
    output = file
    folder_path = os.path.join(output, subfolder)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        
    # Вычисление разницы между значениями в столбцах "Event Time" и "Install Time" в днях
    df['Time Difference (Days)'] = (df['Event Time'] - df['Install Time']).dt.days

    # Фильтрация данных по заданному значению в днях
    filtered_df = df[df['Time Difference (Days)'] < filter_value_days]


    # Проверяем наличие столбца 'AppsFlyer ID' в DataFrame in_app_events_result
    if 'AppsFlyer ID' in filtered_df.columns:


        drop_duplicates_ = filtered_df.drop_duplicates(subset=['AppsFlyer ID'])
        
        # Группируем данные
        group_unique = drop_duplicates_.groupby(['Platform','Media Source', 'Country Code'])['Attributed Touch Type'].count().reset_index()
        group = filtered_df.groupby(['Platform','Media Source', 'Country Code'])['Attributed Touch Type'].count().reset_index()
        
        # Сохраняем результаты в новые xlsx-файлы
        with pd.ExcelWriter(output_path) as writer:
            # Сохраняем данные без мошеннических идентификаторов на первом листе
            filtered_df.to_excel(writer, sheet_name='in_app_events_result', index=False)

            
            # Сохраняем результат группировки на втором листе
            group_unique.to_excel(writer, sheet_name='Grouped Data (unique)', index=False)
            group.to_excel(writer, sheet_name='Grouped Data', index=False)  
            
    else:
        print("No 'Platform...' column in 'in_app_events_result' DataFrame")



# Вызов функции
file_name = '09.2024/Migcredit_09_2024'
file = f'G:/.shortcut-targets-by-id/1-0y0YvUdxNOM5ZnhM4sE0iJVter1OKIh/тест/AF/{file_name}/result_data'
input_csv_path = f'G:/.shortcut-targets-by-id/1-0y0YvUdxNOM5ZnhM4sE0iJVter1OKIh/тест/AF/{file_name}/result_data/in_app_events_result_no_fraud.xlsx'
output_csv_path = f'G:/.shortcut-targets-by-id/1-0y0YvUdxNOM5ZnhM4sE0iJVter1OKIh/тест/AF/{file_name}/result_data/WA/WA.xlsx'
filter_value_days = 30  # Задайте значение в днях

process_csv(input_csv_path, output_csv_path, filter_value_days, file)



NameError: name 'pd' is not defined

In [6]:
#обработка разовая агрегированные
def integration_date(file):
    path = f'{file}'
    filenames = glob.glob(path + "/*.csv")
    
    # Создаем два списка для каждой категории
    agg = []

    
    # Используем эти переменные в начальных значениях
    agg_result = pd.DataFrame()


    # Проверка существования папки, если нет, то создаем
    subfolder = "result_data"
    output_path = os.path.join(file, subfolder)  # Обновленный путь для сохранения файлов
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    # Проходим по всем файлам и добавляем их данные в соответствующий список
    for filename in filenames:
        df = pd.read_csv(filename)
        agg.append(df)
    
    try:
        # Используем pd.concat для каждой категории
        if agg:
            agg_result = pd.concat(agg, axis=0, ignore_index=True)
        else:
            raise pd.errors.EmptyDataError("No 'agg' data to concatenate")
    except pd.errors.EmptyDataError as e:
        print(e)
 
    print(len(agg_result))
    
    # Сохраняем результаты в новые xlsx-файлы
    with pd.ExcelWriter(f"{output_path}/agg.xlsx") as writer:
        # Сохраняем данные без мошеннических идентификаторов на первом листе
        agg_result.to_excel(writer, sheet_name='agg_result', index=False)


name_file = "G:/.shortcut-targets-by-id/1-0y0YvUdxNOM5ZnhM4sE0iJVter1OKIh/тест/AF/08.2024/Fonbet_for_nast_agg"
integration_date(name_file)



NameError: name 'glob' is not defined

In [7]:
#retargeting
def integration_date(file):
    path = f'{file}'
    filenames = glob.glob(path + "/*.csv")

    # Создаем два списка для каждой категории
    in_app_events_dfs = []
    retargeting_inapps_dfs = []

    # Используем эти переменные в начальных значениях
    in_app_events_result = pd.DataFrame()
    retargeting_inapps_result = pd.DataFrame()

    # Проходим по всем файлам и добавляем их данные в соответствующий список
    for filename in filenames:
        if 'in-app-events' in filename:
            df = pd.read_csv(filename)
            in_app_events_dfs.append(df)
        elif 'non-organic' in filename:
            df = pd.read_csv(filename)
            retargeting_inapps_dfs.append(df)

    try:
        # Используем pd.concat для каждой категории
        in_app_events_result = pd.concat(in_app_events_dfs, axis=0, ignore_index=True)
    except pd.errors.EmptyDataError as e:
        print(e)

    try:
        if retargeting_inapps_dfs:
            retargeting_inapps_result = pd.concat(retargeting_inapps_dfs, axis=0, ignore_index=True)
        else:
            raise pd.errors.EmptyDataError("No 'retargeting-inapps' data to concatenate")
    except pd.errors.EmptyDataError as e:
        print(e)
    print(len(in_app_events_result))
    print(len(retargeting_inapps_result))
    # Сохраняем результаты в новые xlsx-файлы
    in_app_events_result.to_excel(f"{file}/result_data/in_app_events_result.xlsx", index=False)
    retargeting_inapps_result.to_excel(f"{file}/result_data/retargeting_inapps_result.xlsx", index=False)

    # Используем lambda-функцию для извлечения значения 'af_price' и создания нового столбца
    in_app_events_result['af_price'] = in_app_events_result['Event Value'].apply(lambda x: float(ast.literal_eval(x).get('af_price', 0)))
    retargeting_inapps_result['af_price'] = retargeting_inapps_result['Event Value'].apply(lambda x: float(ast.literal_eval(x).get('af_price', 0)))

    # Сохраняем результаты в новые xlsx-файлы
    in_app_events_result.to_excel(f"{file}/result_data/in_app_events_result.xlsx", index=False)
    retargeting_inapps_result.to_excel(f"{file}/result_data/retargeting_inapps_result.xlsx", index=False)




    # in_app    
    #Группируем данные и фильтруем их
    # Для af_purchase
    G_sum_by_in_app_P = in_app_events_result[in_app_events_result['Event Name'] == 'af_purchase'].groupby(['Platform', 'Media Source']).agg({'af_price': 'sum', 'Attributed Touch Type': 'count'}).reset_index()
    # Для af_refund
    G_sum_by_in_app_R = in_app_events_result[in_app_events_result['Event Name'] == 'af_refund'].groupby(['Platform', 'Media Source']).agg({'af_price': 'sum', 'Attributed Touch Type': 'count'}).reset_index()




    
    # Объединяем данные по 'Platform' и 'Media Source' для 'af_purchase' и 'af_refund'
    G_in_app_merged_data = pd.merge(G_sum_by_in_app_P, G_sum_by_in_app_R, on=['Platform', 'Media Source'], suffixes=('_P', '_R'), how='left')

    # Заменяем все пропущенные значения на 0
    G_in_app_merged_data.fillna(0, inplace=True)

    # Вычитаем сумму 'af_price' для 'af_refund' из суммы 'af_price' для 'af_purchase'
    G_in_app_merged_data['Difference'] = abs(G_in_app_merged_data['af_price_P']) - abs(G_in_app_merged_data['af_price_R'])
    # Вычитаем сумму 'af_price' для 'af_refund' из суммы 'Attributed Touch Type_P' для 'Attributed Touch Type_R'
    G_in_app_merged_data['Difference_convers'] = abs(G_in_app_merged_data['Attributed Touch Type_P']) - abs(G_in_app_merged_data['Attributed Touch Type_R'])
    print(G_in_app_merged_data)




    # retargeting   
    #Группируем данные и фильтруем их
    # Для af_purchase
    G_sum_by_retargeting_P = retargeting_inapps_result[retargeting_inapps_result['Event Name'] == 'af_purchase'].groupby(['Platform', 'Media Source']).agg({'af_price': 'sum', 'Attributed Touch Type': 'count'}).reset_index()
    # Для af_refund
    G_sum_by_retargeting_R = retargeting_inapps_result[retargeting_inapps_result['Event Name'] == 'af_refund'].groupby(['Platform', 'Media Source']).agg({'af_price': 'sum', 'Attributed Touch Type': 'count'}).reset_index()
    
    # Объединяем данные по 'Platform' и 'Media Source' для 'af_purchase' и 'af_refund'
    G_retargeting_merged_data = pd.merge(G_sum_by_retargeting_P, G_sum_by_retargeting_R, on=['Platform', 'Media Source'], suffixes=('_P', '_R'), how='left')

    # Заменяем все пропущенные значения на 0
    G_retargeting_merged_data.fillna(0, inplace=True)

    # Вычитаем сумму 'af_price' для 'af_refund' из суммы 'af_price' для 'af_purchase'
    G_retargeting_merged_data['Difference'] = abs(G_retargeting_merged_data['af_price_P']) - abs(G_retargeting_merged_data['af_price_R'])
    # Вычитаем сумму 'af_price' для 'af_refund' из суммы 'Attributed Touch Type_P' для 'Attributed Touch Type_R'
    G_retargeting_merged_data['Difference_convers'] = abs(G_retargeting_merged_data['Attributed Touch Type_P']) - abs(G_retargeting_merged_data['Attributed Touch Type_R'])
    print(G_retargeting_merged_data)    

   
    
    
    # Сохраняем результаты в новые xlsx-файлы
    G_in_app_merged_data.to_excel(f"{file}/result_data/in_app_group_summary.xlsx", index=False)
    G_retargeting_merged_data.to_excel(f"{file}/result_data/retargeting_group_summary.xlsx", index=False)




    
    
    # Создаем столбцы с суммами 'af_price' для каждого уникального значения 'Event Name'
    # in_app
    sum_by_in_app = in_app_events_result.groupby('Event Name')['af_price'].sum().reset_index()
    # retargeting
    sum_by_retargeting = retargeting_inapps_result.groupby('Event Name')['af_price'].sum().reset_index()

    # Создаем столбец 'Difference' как разницу между суммой 'af_purchase' и 'af_refund'
    # in_app
    in_app_sum_purchase = sum_by_in_app.loc[sum_by_in_app['Event Name'] == 'af_purchase', 'af_price'].values[0]
    in_app_sum_refund = sum_by_in_app.loc[sum_by_in_app['Event Name'] == 'af_refund', 'af_price'].values[0]
    in_app_difference = abs(in_app_sum_purchase) - abs(in_app_sum_refund)
    in_app_percent = abs(in_app_sum_refund)/(abs(in_app_sum_purchase)/100)

    # retargeting
    retargeting_sum_purchase = sum_by_retargeting.loc[sum_by_retargeting['Event Name'] == 'af_purchase', 'af_price'].values[0]
    retargeting_sum_refund = sum_by_retargeting.loc[sum_by_retargeting['Event Name'] == 'af_refund', 'af_price'].values[0]
    retargeting_difference = abs(retargeting_sum_purchase) - abs(retargeting_sum_refund)
    retargeting_percent = abs(retargeting_sum_refund)/(abs(retargeting_sum_purchase)/100)

    # Создаем DataFrame с одной строкой
    result_in_app = pd.DataFrame({'sum_purchase': in_app_sum_purchase, 'sum_refund': in_app_sum_refund, 'Difference': in_app_difference, 'procent':in_app_percent}, index=[0])
    result_retargeting = pd.DataFrame({'sum_purchase': retargeting_sum_purchase, 'sum_refund': retargeting_sum_refund, 'Difference': retargeting_difference, 'procent':retargeting_percent}, index=[0])

    # Сохраняем результаты в новые xlsx-файлы
    result_in_app.to_excel(f"{file}/result_data/in_app_summary.xlsx", index=False)
    result_retargeting.to_excel(f"{file}/result_data/retargeting_summary.xlsx", index=False)

name_file = "C:/Users/Artur/Desktop/AF/585_02_2024"
integration_date(name_file)


NameError: name 'glob' is not defined

: 

: 

In [8]:
def integration_date(file):
    path = f'{file}'
    filenames = glob.glob(path + "/*.csv")
    
    # Создаем списки для каждой категории и пустые датафреймы для результатов
    partners_daily_ios = []
    partners_daily_android = []
    fraud_post_inapps_ios = []
    fraud_post_inapps_android = []
    partners_daily_result_ios = pd.DataFrame()
    partners_daily_result_android = pd.DataFrame()
    fraud_post_inapps_result_ios = pd.DataFrame()
    fraud_post_inapps_result_android = pd.DataFrame()
    
    # Проходим по всем файлам и добавляем их данные в соответствующий список
    for filename in filenames:
        if 'in-app-events' in filename:
            if 'id' in filename:
                df = pd.read_csv(filename)
                partners_daily_ios.append(df)
            elif 'ru' in filename:
                df = pd.read_csv(filename)
                partners_daily_android.append(df)
        elif 'fraud-post-inapps' in filename:
            if 'id' in filename:
                df = pd.read_csv(filename)
                fraud_post_inapps_ios.append(df)
            elif 'ru' in filename:
                df = pd.read_csv(filename)
                fraud_post_inapps_android.append(df)
    
    # Объединяем данные для каждой категории
    try:
        partners_daily_result_ios = pd.concat(partners_daily_ios, axis=0, ignore_index=True)
    except pd.errors.EmptyDataError as e:
        print(e)
    
    try:
        partners_daily_result_android = pd.concat(partners_daily_android, axis=0, ignore_index=True)
    except pd.errors.EmptyDataError as e:
        print(e)
    
    try:
        fraud_post_inapps_result_ios = pd.concat(fraud_post_inapps_ios, axis=0, ignore_index=True)
    except pd.errors.EmptyDataError as e:
        print(e)
    
    try:
        fraud_post_inapps_result_android = pd.concat(fraud_post_inapps_android, axis=0, ignore_index=True)
    except pd.errors.EmptyDataError as e:
        print(e)
    
    print(len(partners_daily_result_ios))
    print(len(partners_daily_result_android))
    print(len(fraud_post_inapps_result_ios))
    print(len(fraud_post_inapps_result_android))
    
    # Сохраняем результаты в новые xlsx-файлы
    partners_daily_result_ios.to_excel(f"{file}/result_data/partners_daily_result_ios.xlsx", index=False)
    partners_daily_result_android.to_excel(f"{file}/result_data/partners_daily_result_android.xlsx", index=False)
    fraud_post_inapps_result_ios.to_excel(f"{file}/result_data/fraud_post_inapps_result_ios.xlsx", index=False)
    fraud_post_inapps_result_android.to_excel(f"{file}/result_data/fraud_post_inapps_result_android.xlsx", index=False)

    
    """partners_daily_result_ios.rename(columns={"Media Source (pid)":"Media Source"},inplace=True)
    partners_daily_result_android.rename(columns={"Media Source (pid)":"Media Source"},inplace=True)

    partners_daily_result_ios_by = partners_daily_result_ios.groupby('Media Source')['s2s-cpa-conversion (Unique users)'].sum().reset_index()
    partners_daily_result_android_by = partners_daily_result_android.groupby('Media Source')['s2s-cpa-conversion (Unique users)'].sum().reset_index()
    fraud_post_inapps_result_ios_by = fraud_post_inapps_result_ios.groupby('Media Source')['Attributed Touch Type'].count().reset_index()
    fraud_post_inapps_result_android_by = fraud_post_inapps_result_android.groupby('Media Source')['Attributed Touch Type'].count().reset_index()

    print(partners_daily_result_android_by)
    print(fraud_post_inapps_result_android_by)"""

    partners_daily_result_ios_by = partners_daily_result_ios.groupby('Media Source')['Attributed Touch Type'].count().reset_index()
    partners_daily_result_android_by = partners_daily_result_android.groupby('Media Source')['Attributed Touch Type'].count().reset_index()
    fraud_post_inapps_result_ios_by = fraud_post_inapps_result_ios.groupby('Media Source')['Attributed Touch Type'].count().reset_index()
    fraud_post_inapps_result_android_by = fraud_post_inapps_result_android.groupby('Media Source')['Attributed Touch Type'].count().reset_index()

    print(partners_daily_result_android_by)
    print(fraud_post_inapps_result_android_by)
    
    # Объединяем данные по столбцу 'Media Source'
    merged_data_ios = pd.merge(partners_daily_result_ios_by, fraud_post_inapps_result_ios_by, on='Media Source', how='left')
    merged_data_android = pd.merge(partners_daily_result_android_by, fraud_post_inapps_result_android_by, on='Media Source', how='left')

    merged_data_ios.fillna(0, inplace=True)
    merged_data_android.fillna(0, inplace=True)

    print(merged_data_ios)
    print(merged_data_android)
    
    """merged_data_ios['Attributed Touch Type'] = merged_data_ios['Attributed Touch Type'].astype(int)   
    merged_data_android['Attributed Touch Type'] = merged_data_android['Attributed Touch Type'].astype(int)


    # Вычисляем разность значений
    merged_data_ios['Difference'] = merged_data_ios['s2s-cpa-conversion (Unique users)'] - merged_data_ios['Attributed Touch Type']
    merged_data_android['Difference'] = merged_data_android['s2s-cpa-conversion (Unique users)'] - merged_data_android['Attributed Touch Type']"""

    merged_data_ios['Difference'] = merged_data_ios['Attributed Touch Type_x'] - merged_data_ios['Attributed Touch Type_y']
    merged_data_android['Difference'] = merged_data_android['Attributed Touch Type_x'] - merged_data_android['Attributed Touch Type_y']

    
    # Добавление столбца 'platform' в merged_data_ios
    merged_data_ios.insert(0, 'platform', 'ios')
    print(merged_data_ios)
    # Добавление столбца 'platform' в merged_data_android
    merged_data_android.insert(0, 'platform', 'android')
    print(merged_data_android)

    # Объединение DataFrame'ов
    merged_data = pd.concat([merged_data_ios, merged_data_android], ignore_index=True)
    
    # Сохраняем только столбец 'Media Source' и столбец 'Difference' в новый xlsx-файл
    merged_data.to_excel(f"{file}/result_data/MR/merged_result.xlsx", index=False)

name_file = "C:/Users/Artur/Desktop/AF/Банки_ру_01_2024"
integration_date(name_file)

NameError: name 'glob' is not defined